---
description: Design Your Continual Learning Experiments
---
# Putting All Together

Welcome to the "_Putting All Together_" tutorial of the "_From Zero to Hero_" series. In this part we will summarize the major Avalanche features and how you can put them together for your continual learning experiments.

In [ ]:
!pip install avalanche-lib

## 🛴 A Comprehensive Example

Here we report a complete example of the _Avalanche_ usage:

In [7]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.classic import SplitMNIST
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, \
    loss_metrics, timing_metrics, cpu_usage_metrics, confusion_matrix_metrics, disk_usage_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training.strategies import Naive

scenario = SplitMNIST(n_experiences=5)

# MODEL CREATION
model = SimpleMLP(num_classes=scenario.n_classes)

# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.

# log to Tensorboard
tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('log.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(num_classes=scenario.n_classes, save_image=False,
                             stream=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger]
)

# CREATE THE STRATEGY INSTANCE (NAIVE)
cl_strategy = Naive(
    model, SGD(model.parameters(), lr=0.001, momentum=0.9),
    CrossEntropyLoss(), train_mb_size=500, train_epochs=1, eval_mb_size=100,
    evaluator=eval_plugin)

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in scenario.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(scenario.test_stream))

Starting experiment...
Start of experience:  0
Current Classes:  [6, 7]
-- >> Start of training phase << --
-- Starting training on experience 0 (Task 0) from train stream --
100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [00:15<00:00,  1.57it/s]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 720922.1084
	DiskUsage_MB/train_phase/train_stream/Task000 = 720921.7852
	Loss_Epoch/train_phase/train_stream/Task000 = 1.0135
	Loss_MB/train_phase/train_stream/Task000 = 0.1866
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0329
	Time_Epoch/train_phase/train_stream/Task000 = 15.5306
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8093
	Top1_Acc_MB/train_phase/train_stream/Task000 = 1.0000
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████████████████████████████████

	DiskUsage_Stream/eval_phase/test_stream = 720941.4121
	Loss_Stream/eval_phase/test_stream = 3.5860
	Top1_Acc_Stream/eval_phase/test_stream = 0.2126
Start of experience:  2
Current Classes:  [0, 8]
-- >> Start of training phase << --
-- Starting training on experience 2 (Task 0) from train stream --
100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [00:15<00:00,  1.59it/s]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 720950.4150
	DiskUsage_MB/train_phase/train_stream/Task000 = 720950.0918
	Loss_Epoch/train_phase/train_stream/Task000 = 2.2516
	Loss_MB/train_phase/train_stream/Task000 = 0.4531
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0322
	Time_Epoch/train_phase/train_stream/Task000 = 14.7301
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.4297
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.9562
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start

	DiskUsage_Stream/eval_phase/test_stream = 720968.0879
	Loss_Stream/eval_phase/test_stream = 2.0495
	Top1_Acc_Stream/eval_phase/test_stream = 0.2678
Start of experience:  4
Current Classes:  [2, 4]
-- >> Start of training phase << --
-- Starting training on experience 4 (Task 0) from train stream --
100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [00:15<00:00,  1.50it/s]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 720976.9932
	DiskUsage_MB/train_phase/train_stream/Task000 = 720976.6699
	Loss_Epoch/train_phase/train_stream/Task000 = 2.2955
	Loss_MB/train_phase/train_stream/Task000 = 1.1687
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0367
	Time_Epoch/train_phase/train_stream/Task000 = 15.4365
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.2381
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.9200
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start


## 🤝 Run it on Google Colab

You can run _this chapter_ and play with it on Google Colaboratory: [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ContinualAI/avalanche/blob/master/notebooks/from-zero-to-hero-tutorial/07_putting-all-together.ipynb)